In [1]:
import pandas as pd

from torch.utils.data import Dataset
from torchvision import transforms

from PIL import Image
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import cv2

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/팀우이홍황')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# test before / after 비교용 반복 출력

In [5]:
test_set = pd.read_csv('./train.csv')
test_set.head()

,Unnamed: 0,img_path,meta_path,weights
0,0,./dataset/train\CASE01\image\CASE01_01.png,./dataset/train\CASE01\meta\CASE01_01.csv,49.193
1,1,./dataset/train\CASE01\image\CASE01_02.png,./dataset/train\CASE01\meta\CASE01_02.csv,59.764
2,2,./dataset/train\CASE01\image\CASE01_03.png,./dataset/train\CASE01\meta\CASE01_03.csv,72.209
3,3,./dataset/train\CASE01\image\CASE01_04.png,./dataset/train\CASE01\meta\CASE01_04.csv,85.737
4,4,./dataset/train\CASE01\image\CASE01_05.png,./dataset/train\CASE01\meta\CASE01_05.csv,102.537


In [6]:
test_set.describe()

,Unnamed: 0,weights
count,1592.000000,1592.000000
mean,795.500000,80.838052
std,459.715129,102.864259
min,0.000000,0.000000
25%,397.750000,5.836500
50%,795.500000,35.088500
75%,1193.250000,118.851750
max,1591.000000,481.375000


In [7]:
w_df = test_set.sort_values('weights', ignore_index =True)
w_df

,Unnamed: 0,img_path,meta_path,weights
0,472,./dataset/train\CASE27\image\CASE27_01.jpg,./dataset/train\CASE27\meta\CASE27_01.csv,0.000
1,1491,./dataset/train\CASE71\image\CASE71_05.png,./dataset/train\CASE71\meta\CASE71_05.csv,0.000
2,674,./dataset/train\CASE38\image\CASE38_01.png,./dataset/train\CASE38\meta\CASE38_01.csv,0.000
3,1490,./dataset/train\CASE71\image\CASE71_04.png,./dataset/train\CASE71\meta\CASE71_04.csv,0.000
4,1492,./dataset/train\CASE71\image\CASE71_06.png,./dataset/train\CASE71\meta\CASE71_06.csv,0.000
...,...,...,...,...
1587,565,./dataset/train\CASE33\image\CASE33_25.png,./dataset/train\CASE33\meta\CASE33_25.csv,439.670
1588,592,./dataset/train\CASE34\image\CASE34_27.png,./dataset/train\CASE34\meta\CASE34_27.csv,444.465
1589,540,./dataset/train\CASE32\image\CASE32_26.png,./dataset/train\CASE32\meta\CASE32_26.csv,461.639
1590,619,./dataset/train\CASE35\image\CASE35_27.png,./dataset/train\CASE35\meta\CASE35_27.csv,469.019


In [8]:
w_df['Unnamed: 0'] = [x for x in range(1592)]

In [9]:
w_df[:200 :2]

,Unnamed: 0,img_path,meta_path,weights
0,0,./dataset/train\CASE27\image\CASE27_01.jpg,./dataset/train\CASE27\meta\CASE27_01.csv,0.000
2,2,./dataset/train\CASE38\image\CASE38_01.png,./dataset/train\CASE38\meta\CASE38_01.csv,0.000
4,4,./dataset/train\CASE71\image\CASE71_06.png,./dataset/train\CASE71\meta\CASE71_06.csv,0.000
6,6,./dataset/train\CASE36\image\CASE36_01.png,./dataset/train\CASE36\meta\CASE36_01.csv,0.000
8,8,./dataset/train\CASE12\image\CASE12_01.png,./dataset/train\CASE12\meta\CASE12_01.csv,0.025
...,...,...,...,...
190,190,./dataset/train\CASE30\image\CASE30_05.jpg,./dataset/train\CASE30\meta\CASE30_05.csv,1.748
192,192,./dataset/train\CASE13\image\CASE13_08.png,./dataset/train\CASE13\meta\CASE13_08.csv,1.764
194,194,./dataset/train\CASE57\image\CASE57_05.png,./dataset/train\CASE57\meta\CASE57_05.csv,1.791
196,196,./dataset/train\CASE40\image\CASE40_05.png,./dataset/train\CASE40\meta\CASE40_05.csv,1.800


중량 수치와 매치가 되지 않는 이미지를 찾기 위해 중량 기준 오름차순으로 정렬하여 이미지를 출력시킨 뒤 육안으로 확인

## ~400

In [ ]:

from IPython import display 

rows = 1
cols = 2

# :1000, 1000:2000, 2000:3000, 3000:
for i, j, before, after in zip(w_df['weights'][:401 :2], w_df['weights'][1:400 :2], 
                            w_df["img_path"][:401 :2], w_df['img_path'][1:400 :2]):
    
    b, a = before[-13:], after[-13:]
    display.clear_output(wait=0.2) # 이미지를 지우고 대기시간
    fig = plt.figure(figsize=(16,8)) # 도화지를 만든다

    img1 = cv2.imread(f'{before}')
    img2 = cv2.imread(f'{after}')
    img1 = cv2.resize(img1,(0,0), fx = 0.5, fy = 0.5)
    img2 = cv2.resize(img2,(0,0), fx = 0.5, fy = 0.5)

    ax1 = fig.add_subplot(rows,cols,1) # 서브플롯으로 두 이미지를 한번에 표현
    #ax1.imshow(img1)
    ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) # cv2 특성상 COLOR_BGR2RGB 필요
    ax1.set_title(f"{b} / {i}")
    ax1.axis("off") # 이미지를 보고자 함이므로 축 필요없음

    ax1 = fig.add_subplot(rows,cols, 2)
    #ax1.imshow(img2)
    ax1.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax1.set_title(f"{a} / {j}")
    ax1.axis("off")

    plt.show()


## 401~800

In [ ]:

from IPython import display 

rows = 1
cols = 2

# :1000, 1000:2000, 2000:3000, 3000:
for i, j, before, after in zip(w_df['weights'][400:801 :2], w_df['weights'][401:800 :2], 
                            w_df["img_path"][400:801 :2], w_df['img_path'][401:800 :2]):
    
    b, a = before[-13:], after[-13:]
    display.clear_output(wait=0.2) # 이미지를 지우고 대기시간
    fig = plt.figure(figsize=(16,8)) # 도화지를 만든다

    img1 = cv2.imread(f'{before}')
    img2 = cv2.imread(f'{after}')
    img1 = cv2.resize(img1,(0,0), fx = 0.5, fy = 0.5)
    img2 = cv2.resize(img2,(0,0), fx = 0.5, fy = 0.5)

    ax1 = fig.add_subplot(rows,cols,1) # 서브플롯으로 두 이미지를 한번에 표현
    #ax1.imshow(img1)
    ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) # cv2 특성상 COLOR_BGR2RGB 필요
    ax1.set_title(f"{b} / {i}")
    ax1.axis("off") # 이미지를 보고자 함이므로 축 필요없음

    ax1 = fig.add_subplot(rows,cols, 2)
    #ax1.imshow(img2)
    ax1.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax1.set_title(f"{a} / {j}")
    ax1.axis("off")

    plt.show()

## 801~1200

In [ ]:

from IPython import display 

rows = 1
cols = 2

# :1000, 1000:2000, 2000:3000, 3000:
for i, j, before, after in zip(w_df['weights'][800:1201 :2], w_df['weights'][801:1200 :2], 
                            w_df["img_path"][800:1201 :2], w_df['img_path'][801:1200 :2]):
    
    b, a = before[-13:], after[-13:]
    display.clear_output(wait=0.2) # 이미지를 지우고 대기시간
    fig = plt.figure(figsize=(16,8)) # 도화지를 만든다

    img1 = cv2.imread(f'{before}')
    img2 = cv2.imread(f'{after}')
    img1 = cv2.resize(img1,(0,0), fx = 0.5, fy = 0.5)
    img2 = cv2.resize(img2,(0,0), fx = 0.5, fy = 0.5)

    ax1 = fig.add_subplot(rows,cols,1) # 서브플롯으로 두 이미지를 한번에 표현
    #ax1.imshow(img1)
    ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) # cv2 특성상 COLOR_BGR2RGB 필요
    ax1.set_title(f"{b} / {i}")
    ax1.axis("off") # 이미지를 보고자 함이므로 축 필요없음

    ax1 = fig.add_subplot(rows,cols, 2)
    #ax1.imshow(img2)
    ax1.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax1.set_title(f"{a} / {j}")
    ax1.axis("off")

    plt.show()


## 1200~

In [ ]:

from IPython import display 

rows = 1
cols = 2

# :1000, 1000:2000, 2000:3000, 3000:
for i, j, before, after in zip(w_df['weights'][1200: :2], w_df['weights'][1201: :2], 
                            w_df["img_path"][1200: :2], w_df['img_path'][1201: :2]):
    
    b, a = before[-13:], after[-13:]
    display.clear_output(wait=0.2) # 이미지를 지우고 대기시간
    fig = plt.figure(figsize=(16,8)) # 도화지를 만든다

    img1 = cv2.imread(f'{before}')
    img2 = cv2.imread(f'{after}')
    img1 = cv2.resize(img1,(0,0), fx = 0.5, fy = 0.5)
    img2 = cv2.resize(img2,(0,0), fx = 0.5, fy = 0.5)

    ax1 = fig.add_subplot(rows,cols,1) # 서브플롯으로 두 이미지를 한번에 표현
    #ax1.imshow(img1)
    ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)) # cv2 특성상 COLOR_BGR2RGB 필요
    ax1.set_title(f"{b} / {i}")
    ax1.axis("off") # 이미지를 보고자 함이므로 축 필요없음

    ax1 = fig.add_subplot(rows,cols, 2)
    #ax1.imshow(img2)
    ax1.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax1.set_title(f"{a} / {j}")
    ax1.axis("off")

    plt.show()

## 이상치 리스트

In [1]:
D_Outlier = ['CASE36_01','CASE39_01','CASE37_01','CASE26_01','CASE11_01','CASE49_03',
             'CASE22_07','CASE61_02','CASE22_09','CASE22_10','CASE56_11','CASE57_10',
             'CASE18_09','CASE22_11','CASE60_34','CASE60_33','CASE60_32','CASE73_34',
             'CASE60_31','CASE60_30','CASE60_29','CASE60_28','CASE60_27','CASE26_45',
             'CASE26_46','CASE60_26','CASE70_33','CASE60_25','CASE26_44','CASE26_47',
             'CASE26_41','CASE26_43','CASE26_42','CASE26_40','CASE26_35','CASE73_31',
             'CASE23_02','CASE51_21','CASE73_14','CASE73_30','CASE71_30','CASE69_19',
             'CASE73_01','CASE73_10','CASE71_31','CASE45_17','CASE70_24','CASE70_23',
             'CASE23_03','CASE70_22','CASE70_21','CASE23_04','CASE23_08','CASE23_06',
             'CASE23_07','CASE23_05','CASE23_09','CASE17_02','CASE09_08'] # 뚜렷한 이상치 => 학습 시 제외

N_Outlier = ['CASE52_01','CASE57_19','CASE69_20','CASE02_10','CASE70_20','CASE70_18',
             'CASE70_03','CASE04_09','CASE70_17','CASE70_11','CASE70_12','CASE70_13'] # 모호한 이상치 => 포함 or 제외 둘 중 더 점수가 좋은 쪽을 선택